# Transparency / Spending Slide Explorer

Testing a recipe for generating 1.5D slide show summaries of transparency spending data, as accounting for eg 80% of spend in each particular area.

In [1]:
#Download transparency spending data file
#In this case, h/t @owenboswarva
url='http://www.datadaptive.com/misc/CO_SPEND_201004_201611.xlsx'
#!wget -P data/ $url

--2017-01-23 14:50:53--  http://www.datadaptive.com/misc/CO_SPEND_201004_201611.xlsx
Resolving www.datadaptive.com... 192.185.39.75
Connecting to www.datadaptive.com|192.185.39.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 952091 (930K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: 'data/CO_SPEND_201004_201611.xlsx.6'

CO_SPEND_201004_201 100%[=====================>] 929.78K   466KB/s   in 2.0s   

2017-01-23 14:50:55 (466 KB/s) - 'data/CO_SPEND_201004_201611.xlsx.6' saved [952091/952091]



In [2]:
#Use the pandas package form wrangling tabular data
import pandas as pd

In [3]:
#Read in the data file and do a bit of cleaning
df=pd.read_excel('data/{}'.format(url.split('/')[-1]), thousands=',')
df.head(3)

,Departmental Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount £,Description,Supplier Postcode,Supplier Type,Expenditure Type
0,Cabinet Office,CABINET OFFICE,2010-04-01,Other Staff Related Costs,CIVIL SERVICE CAPABILITY GROUP,PARITY SOLUTIONS LTD,3000009169,75416.35,Fast Stream Recruitment,NaN,NaN,NaN
1,Cabinet Office,CABINET OFFICE,2010-04-01,Consultancy,CIVIL SERVICE PENSIONS,DELOITTE MCS LTD,3000009144,31530.25,Professional Services: Programme support work ...,NaN,NaN,NaN
2,Cabinet Office,CABINET OFFICE,2010-04-01,Supplies and Services,CO SHARED SERVICES,DWP NORCROSS,3000009146,133333.33,"Provision of Accountancy Services, Employee Se...",NaN,NaN,NaN


In [4]:
#DATA SETTINGS
datecol='Date'

org='Cabinet Office'

#The following settings are used to define columns that chunk the report
top='Entity' #A high level split - eg capital vs revenue
amount='Amount £' #The spending amount column
majordivision='Description' #Major grouping eg council directorate
supplier='Supplier' #The supplier name column
repTyps=['Expense Type','Expense Area'] #column groupings to generate supplier reports form

In [5]:
#USER SETTINGS


#Report period - for now, focus on monthly reporting
year=2016
month=9

#One of the aims of the report is to be able to account for eg 80% of the spend.
#(Pareto principle sugggests this will be covered by eg 20% of contributors.)
#Eg report that accounts for eg 80% of total spend, plus any items that account for more than 10% of total
accountForPC=80
hedgebetsPC=0
minPC=10

In [6]:
##TRY TO MAKE EVERYTHING BELOW HERE ABSTRACT

In [7]:
#Quick clean
for col in df.columns:
    #Pandas natively has 'O' (object) type for string columns
    if col==amount:
        df[amount]=df[amount].astype(float)
    else:
        df[col]=df[col].astype(str)
        df[col]=df[col].str.strip()
    if col==datecol:
        df[datecol]=pd.to_datetime(df[datecol])
df.head(3)

,Departmental Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount £,Description,Supplier Postcode,Supplier Type,Expenditure Type
0,Cabinet Office,CABINET OFFICE,2010-04-01,Other Staff Related Costs,CIVIL SERVICE CAPABILITY GROUP,PARITY SOLUTIONS LTD,3000009169,75416.35,Fast Stream Recruitment,nan,nan,nan
1,Cabinet Office,CABINET OFFICE,2010-04-01,Consultancy,CIVIL SERVICE PENSIONS,DELOITTE MCS LTD,3000009144,31530.25,Professional Services: Programme support work ...,nan,nan,nan
2,Cabinet Office,CABINET OFFICE,2010-04-01,Supplies and Services,CO SHARED SERVICES,DWP NORCROSS,3000009146,133333.33,"Provision of Accountancy Services, Employee Se...",nan,nan,nan


In [8]:
#Generate reporting period dataset
df_base=df[(df[datecol].dt.year==year) & (df[datecol].dt.month==month)]
df_base=df_base.dropna(axis=1,how='all')
df_base.head(2)

,Departmental Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount £,Description,Supplier Postcode,Supplier Type,Expenditure Type
11557,CABINET OFFICE,CABINET OFFICE,2016-09-01,PROFESSIONAL ADVICE & OPINION VAT RECOVERABLE,HUB MEMPHIS,EVERSHEDS LLP,3000075785,70000.00,PROFESSIONAL PROPERTY ADVICE,nan,nan,nan
11558,CABINET OFFICE,CABINET OFFICE,2016-09-01,SERVICE PAYMENTS,STERIA SHARED SERVICES,SHARED SERVICES CONNECTED LTD,3000075788,476500.34,SERVICE PAYMENT,nan,nan,nan


## Generate Slide Deck

Slide deck is generated using Jupyter notebook (slide format) as an intermediate representation and then generating *reveal.js* slide deck from that.

More info: [A Recipe for Automatically Going From Data to Text to Reveal.js Slides](https://blog.ouseful.info/2017/01/20/a-recipe-for-automatically-going-from-data-to-text-to-reveal-js-slides/).

In [9]:
import nbformat as nb
import nbformat.v4.nbbase as nb4

def addSlideComponent(notebook, content, styp=''):
    ''' Function to add a slide component to a slide notebook '''
    if styp in ['slide','fragment','subslide']: styp= {"slideshow": {"slide_type": styp }}
    else: styp={}
    notebook.cells.append(nb4.new_markdown_cell(content, metadata=styp))

### Formatting Helper Functions

In [10]:
def fa(amount):
    ''' Format a cash amount '''
    return "£{:,.2f}".format(amount)

def fpc(amount):
    ''' Format a percentage '''
    return "{:.2f}%".format(amount)

### Reporting Helper Functions

One of the aims of the report is to be able to account for eg 80% of the spend. (Pareto principle sugggests this will be covered by eg 20% of contributors.)

In [11]:
def paretoXY(s, x, y,threshold=0):
    ''' Return a list until you account for X% of the whole and remainders are less than Y% individually.
        The threshold percentage allows you to hedge your bets and check items just past the treshold. '''
    #Generate percentages, and sort, and find accumulated total
    #Note that there may be negative payments which means the cumulative percentage can go > 100%
    df=pd.DataFrame( s[s>0].sort_values(ascending=False) )
    df['pc']= 100*s/s.sum()
    df['cum_pc']=df['pc'].cumsum()
    #Threshold
    return df[ (df['cum_pc']-df['pc'] <= x+ x*threshold/100) | (df['pc'] >= y-y*threshold/100) ]

### Slide Notebook Generator

Routines for automatically constructing the intermediate slide notebook.

In [12]:
test=nb4.new_notebook()

In [13]:
def topGroup(df,top):
    return df.groupby(top).sum()

In [14]:
topgroup=topGroup(df_base,top)

overalltotal=topgroup.sum()[amount]

In [15]:
#INTRO SLIDE
addSlideComponent(test, '# {} Monthly Spending Report - {}/{} '.format(org,month,year),'slide')

txt1=['The total spend for the month was __{total}__.\n\n This was split:\n'.format(total=fa(overalltotal))]

for typ in topgroup.index.values:
    txt1.append('- __{typval}__ ({typPC}) on {typ} items\n'.format(typ=typ,
                                                                   typval=fa(topgroup.ix[typ][amount]),
                                                                   typPC=fpc(100*topgroup.ix[typ][amount]/topgroup[amount].sum())))

addSlideComponent(test, txt1)

In [16]:
majordivSpend=df_base.groupby(majordivision).sum().sort_values(amount,ascending=False)
majordivTopSpend=df_base.groupby([majordivision,top]).sum()

totalpc=df_base.pivot_table(index=[majordivision],values=amount,aggfunc=sum).sort_values(ascending=False)
totalpc=100*totalpc/totalpc.sum()

#Find percentages by top
tmp=df_base.pivot_table(index=[majordivision,top],values=amount,aggfunc=sum).sort_values(ascending=False)
pc=tmp.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

In [17]:
#MAJOR DIVISION SLIDE (VERTICAL STACK)
addSlideComponent(test,'## Monthly Spend by {}'.format(majordivision),'slide')

addSlideComponent(test,'The spend for each {} was as follows:'.format(majordivision))

for x,y in majordivSpend.iterrows():
    addSlideComponent(test, '- {}\n  - total spend: {} ({} of total monthly spend)\n'.format(x,
                                                                                              fa(y[amount]),
                                                                                              fpc(totalpc.ix[x])),'fragment')

In [18]:
#Deeper Dive Grouped by Columns of Interest...
for majordiv in majordivSpend.index:
    
    addSlideComponent(test,'## Major Monthly Spends  - {}'.format(majordiv),'slide')
    addSlideComponent(test, 'Total spend: {} ({} of total {} monthly spend)\n'.format(fa(majordivSpend.ix[majordiv][amount]),
                                                                                      fpc(totalpc.ix[majordiv]),
                                                                                      org))
    for rc in topgroup.index.values:
        try:
            txt='({} of {} total, {} of {} total)'.format(fpc(pc.ix[majordiv,rc]),majordivision,
                                                          fpc(100*majordivTopSpend.ix[majordiv,rc][amount]/overalltotal),
                                                          org)
            addSlideComponent(test,'- {}: {} {}'.format(rc,
                                                         fa(majordivTopSpend.ix[majordiv,rc][amount]),
                                                         txt))
        except: pass
    
    for typ in repTyps:
        directorateRCTyp=df_base.pivot_table(index=[majordivision,top,typ],
                                             values=amount,aggfunc=sum)
        
        addSlideComponent(test,'## Major Monthly Spend by {}: {}'.format(typ,majordiv),'slide')

        for rc in topgroup.index.values:
            try:
                txt='({} of {} total, {} of {} total)'.format(fpc(pc.ix[majordiv,rc]),majordivision,
                                                              fpc(100*majordivTopSpend.ix[majordiv,rc][amount]/overalltotal),
                                                              org)
                addSlideComponent(test,'### {} {}'.format(rc,txt),'fragment')

                thisDirectorateRC=paretoXY(directorateRCTyp.ix[(majordiv,rc)].sort_values(ascending=False),
                                                                   accountForPC,
                                                                   minPC,
                                                                   threshold=hedgebetsPC)

                for x,y in thisDirectorateRC.iterrows():
                    x=x.strip()
                    txt='- {}: {} ({} of corresponding {} {} spend)'.format(x,
                                                                                      fa(y[amount]),
                                                                                      fpc(y['pc']),majordivision,
                                                                                      rc.lower())
                    addSlideComponent(test, txt)
            except: pass

        #Now do subslides for major spend within those areas by supplier
        try:
            for rc in topgroup.index.values:
                thisDirectorateRC=paretoXY(directorateRCTyp.ix[(majordiv,rc)].sort_values(ascending=False),
                                                               accountForPC,
                                                               minPC,
                                                               threshold=hedgebetsPC)
                for x,y in thisDirectorateRC.iterrows():
                    tmp=df_base[(df_base[majordivision]==majordiv) & (df_base[typ]==x)]
                    directorateRCTypSupplier=tmp.pivot_table(index=[top,
                                                                              supplier],
                                                                       values=amount, aggfunc=sum)
                    thisDirectorateRCSupp=paretoXY(directorateRCTypSupplier.ix[rc].sort_values(ascending=False),
                                                                   accountForPC,
                                                                   minPC,
                                                                   threshold=hedgebetsPC)

                    addSlideComponent(test,'## {} - {} ({})  Major Suppliers'.format(majordiv,x,rc),'subslide')
                    txt='Overall, this {} accounts for __{}__ ({} of corresponding {} {} spend)'.format(typ,fa(y[amount]),
                                                                                   fpc(y['pc']),majordivision,
                                                                                   rc.lower())
                    addSlideComponent(test, txt)

                    for p,q in thisDirectorateRCSupp.iterrows():
                        p=p.strip()
                        txt='- {}: {} ({} of corresponding spend)'.format(p,
                                                                          fa(q[amount]),
                                                                          fpc(q['pc']))
                        addSlideComponent(test, txt)                  
        except: pass

### Generate Slide Deck and Display via Local Server

In [19]:
nbf='testfullslidenotebook_caboffice.ipynb'
nb.write(test,nbf)
!jupyter nbconvert $nbf --to slides --post serve

[NbConvertApp] Converting notebook testfullslidenotebook_caboffice.ipynb to slides
[NbConvertApp] Writing 546115 bytes to testfullslidenotebook_caboffice.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.1.0
Serving your slides at http://127.0.0.1:8000/testfullslidenotebook_caboffice.slides.html
Use Control-C to stop this server
^C

Interrupted
